# Prep data

In [ ]:
%sh git clone http://github.com/tensorflow/models.git

In [ ]:
cd models/research/

In [ ]:
%sh apt-get -y install protobuf-compiler

In [ ]:
%sh protoc object_detection/protos/*.proto --python_out=.

In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
pip install .

In [ ]:
cd /databricks/driver

In [ ]:
%sh
set -x
mkdir -p pet_data
cd pet_data
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
tar -xf images.tar.gz
tar -xf annotations.tar.gz
ls pet_data

In [ ]:
%sh
# Note: It is normal to see some warnings when running this script. You may ignore them.
cd /databricks/driver/models/research
mkdir -p /dbfs/mnt/ext-infy-engagement/Data/object_detection/resnet/data
python object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=object_detection/data/pet_label_map.pbtxt \
    --data_dir=/databricks/driver/pet_data \
    --output_dir=/dbfs/mnt/ext-infy-engagement/Data/object_detection/resnet/data/
#--output_dir=/databricks/driver/object_detection/resnet/data

In [ ]:
%sh 
mkdir -p /databricks/driver/object_detection/resnet/data
cp -r /dbfs/mnt/ext-infy-engagement/Data/object_detection/resnet/data/* /databricks/driver/object_detection/resnet/data
cp object_detection/data/pet_label_map.pbtxt /databricks/driver/object_detection/resnet/data/label_map.txt

In [ ]:
%sh
mkdir -p /databricks/driver/object_detection/resnet/test/
mv /databricks/driver/object_detection/resnet/data/pet_faces* /databricks/driver/object_detection/resnet/test/
cp /databricks/driver/object_detection/resnet/test/pet_faces_train.record-00000-of-00010 /databricks/driver/object_detection/resnet/data/
cp /databricks/driver/object_detection/resnet/test/pet_faces_val.record-00000-of-00010 /databricks/driver/object_detection/resnet/data/
ls /databricks/driver/object_detection/resnet/data/

# Get model and create job config

In [ ]:
%sh
set -x
model=faster_rcnn_resnet50_v1_640x640_coco17_tpu-8
test -e $model.tar.gz || curl -fsO http://download.tensorflow.org/models/object_detection/tf2/20200711/$model.tar.gz
tar -xvf $model.tar.gz --no-same-owner 
cp $model/checkpoint/* /databricks/driver/object_detection/resnet/data
ls /databricks/driver/object_detection/resnet/data | grep ck

In [ ]:
%sh
curl -o object_detection.config https://raw.githubusercontent.com/Srigandhan/TF/main/object_detection.config
cp object_detection.config /databricks/driver/object_detection/resnet/object_detection.config
cat /databricks/driver/object_detection/resnet/object_detection.config

# Config and run job
Run the following cell and open the View TensorBoard link to monitor progress.

In [ ]:
%sh
curl -o model_main_tf2_new.py https://raw.githubusercontent.com/Srigandhan/TF/main/model_main_tf2_new.py
cp model_main_tf2_new.py models/research/object_detection/model_main_tf2.py
cat models/research/object_detection/model_main_tf2.py

In [ ]:
%sh
cd /databricks/driver/models/research/
python object_detection/model_main_tf2.py --logtostderr --model_dir=/databricks/driver/object_detection/resnet/models --pipeline_config_path=/databricks/driver/object_detection/resnet/object_detection.config

In [ ]:
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
print(token)
dbutils.fs.ls("file:///root/")
#dbutils.fs.put("file:///root/.databrickscfg","[DEFAULT]\nhost=https://community.cloud.databricks.com\ntoken = "+token,overwrite=True)